In [147]:
import pandas as pd
import numpy as np

## Load data

Load data with multiindex headers from rows 0 and 1

In [74]:
# only import column names
df_for_column_names = pd.read_excel(
    "./data/raw/Receptna_lista+2010-2019_i.xlsx", 
    sheet_name="ZDRAVILA_OZZ-2010-2019",
    header=[1],
    nrows=0
)
df_for_column_names.columns[0:11]

In [207]:
# this import is for data and has multiindex
df = pd.read_excel(
    "./data/raw/Receptna_lista+2010-2019_i.xlsx", 
    sheet_name="ZDRAVILA_OZZ-2010-2019",
    header=[0,1],
    na_values=["", " ", "-"]
)
df.info()

In [209]:
# set index
df.set_index(keys=df.columns[0:11].to_list(), inplace=True)
df.head()

## Data processing

Drop 1st row (magistralna zdravila)

In [212]:
# first row is irrelevant, let's drop it
df.drop(axis=0, index=0, inplace=True)
df.head()

**Stack year columns**

In [366]:
# we need to stack year columns for all final values
df_stacked = df.stack(1)

In [367]:
df_stacked.reset_index(inplace=True)
df_stacked.head()

In [373]:
# we need to repair column names

new_columns = df_stacked.columns.to_list()
new_columns[0:11] = df_for_column_names.columns[0:11]

new_columns[11] = "leto"
df_stacked.columns = new_columns

df_stacked.head()

# Explore for missing data, extreme values and dtypes

In [374]:
df_stacked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39010 entries, 0 to 39009
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   NacionalnaŠifra            39010 non-null  int64  
 1   ImeZdravila                37150 non-null  object 
 2   PoimenovanjeZdravila       39010 non-null  object 
 3   KratkoImeZdravila          39010 non-null  object 
 4   SplosnoImeZdravila         39010 non-null  object 
 5   AtcOznaka                  38970 non-null  object 
 6   KolicinaOEZzaAplikacijo    39010 non-null  float64
 7   OznakaOEZ                  39010 non-null  object 
 8   DDD                        32310 non-null  float64
 9   DDD_OznakaOEZ              32310 non-null  object 
 10  Število_DDD_v_pakiranju    32310 non-null  float64
 11  leto                       39010 non-null  object 
 12  Celotna vrednost receptov  39010 non-null  float64
 13  Stevilo DDD                39010 non-null  flo

In [379]:
# change type of amount of dosage unit for application

df_stacked.KolicinaOEZzaAplikacijo =  df_stacked.KolicinaOEZzaAplikacijo.astype("int64")
df_stacked.describe()

,NacionalnaŠifra,KolicinaOEZzaAplikacijo,DDD,Število_DDD_v_pakiranju,Celotna vrednost receptov,Stevilo DDD,Stevilo DID,Vrednost OZZ,Število receptov,Število škatel
count,39010.000000,39010.000000,3.231000e+04,32310.000000,3.901000e+04,3.901000e+04,39010.000000,3.901000e+04,39010.000000,3.901000e+04
mean,105546.598821,35.093822,1.205300e+04,30.829208,1.165494e+05,2.387694e+05,0.317418,7.330823e+04,4205.952679,9.847010e+03
std,107418.845126,43.594916,2.263497e+05,58.640488,3.646566e+05,1.140951e+06,1.517293,2.749483e+05,17188.083086,4.058745e+04
min,27.000000,0.000000,9.000000e-04,0.200000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,38490.000000,7.000000,2.500000e+00,11.200000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
50%,88889.000000,28.000000,2.000000e+01,28.000000,4.683640e+03,1.440000e+02,0.000000,1.655455e+03,62.000000,1.475000e+02
75%,146128.000000,35.000000,2.000000e+02,30.000000,7.964545e+04,6.428200e+04,0.090000,3.634502e+04,1989.500000,4.786500e+03
max,630020.000000,800.000000,9.000000e+06,2443.181800,1.300312e+07,5.033640e+07,66.970000,1.011609e+07,501404.000000,1.374125e+06


In [384]:
# check unusually large DDD count per product pack

above_90_quantile_DDD_in_pack = df_stacked.loc[df_stacked.Število_DDD_v_pakiranju > df_stacked.Število_DDD_v_pakiranju.quantile(q=0.9)]
above_90_quantile_DDD_in_pack.sort_values(by="Število_DDD_v_pakiranju", ascending=False).head()

,NacionalnaŠifra,ImeZdravila,PoimenovanjeZdravila,KratkoImeZdravila,SplosnoImeZdravila,AtcOznaka,KolicinaOEZzaAplikacijo,OznakaOEZ,DDD,DDD_OznakaOEZ,Število_DDD_v_pakiranju,leto,Celotna vrednost receptov,Stevilo DDD,Stevilo DID,Vrednost OZZ,Število receptov,Število škatel
27433,144584,ELMEX,"ELMEX GELEE 1,25 % dentalni gel","ELMEX GELEE 1,25 % dentalni gel 215 g","olaflur, dektaflur in natrijev fluorid",A01AA51,215,g,1.1,mg,2443.1818,2013,0.0,0.0,0.0,0.0,0,0
27435,144584,ELMEX,"ELMEX GELEE 1,25 % dentalni gel","ELMEX GELEE 1,25 % dentalni gel 215 g","olaflur, dektaflur in natrijev fluorid",A01AA51,215,g,1.1,mg,2443.1818,2015,0.0,0.0,0.0,0.0,0,0
27436,144584,ELMEX,"ELMEX GELEE 1,25 % dentalni gel","ELMEX GELEE 1,25 % dentalni gel 215 g","olaflur, dektaflur in natrijev fluorid",A01AA51,215,g,1.1,mg,2443.1818,2016,0.0,0.0,0.0,0.0,0,0
27432,144584,ELMEX,"ELMEX GELEE 1,25 % dentalni gel","ELMEX GELEE 1,25 % dentalni gel 215 g","olaflur, dektaflur in natrijev fluorid",A01AA51,215,g,1.1,mg,2443.1818,2012,0.0,0.0,0.0,0.0,0,0
27431,144584,ELMEX,"ELMEX GELEE 1,25 % dentalni gel","ELMEX GELEE 1,25 % dentalni gel 215 g","olaflur, dektaflur in natrijev fluorid",A01AA51,215,g,1.1,mg,2443.1818,2011,0.0,0.0,0.0,0.0,0,0


In [385]:
# check unusually large DDD 

above_90_quantile_DDD = df_stacked.loc[df_stacked.DDD > df_stacked.DDD.quantile(q=0.9)]
above_90_quantile_DDD.sort_values(by="DDD", ascending=False).head()

,NacionalnaŠifra,ImeZdravila,PoimenovanjeZdravila,KratkoImeZdravila,SplosnoImeZdravila,AtcOznaka,KolicinaOEZzaAplikacijo,OznakaOEZ,DDD,DDD_OznakaOEZ,Število_DDD_v_pakiranju,leto,Celotna vrednost receptov,Stevilo DDD,Stevilo DID,Vrednost OZZ,Število receptov,Število škatel
24531,126977,COLOMYCIN,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,kolistin,J01XB01,100,ml,9000000.0,i.e.,1.1111,2011,0.00,0.00,0.0,0.00,0,0
24532,126977,COLOMYCIN,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,kolistin,J01XB01,100,ml,9000000.0,i.e.,1.1111,2012,0.00,0.00,0.0,0.00,0,0
24533,126977,COLOMYCIN,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,kolistin,J01XB01,100,ml,9000000.0,i.e.,1.1111,2013,10453.29,105.55,0.0,9199.48,14,95
24534,126977,COLOMYCIN,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,kolistin,J01XB01,100,ml,9000000.0,i.e.,1.1111,2014,35854.23,362.22,0.0,32160.36,37,326
24535,126977,COLOMYCIN,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,COLOMYCIN INJECTION 1.000.000 IE prašek za raz...,kolistin,J01XB01,100,ml,9000000.0,i.e.,1.1111,2015,38389.55,387.77,0.0,35122.30,48,349


Since everything seems to be ok with extreme values, we can proceede further.

## Add ATC 1-5 descriptions

We need to add descriptive names of ATC groups so we can compare groups of substances

In [380]:
# import ATC description datasets
df_atc1 = pd.read_csv("data/processed/ATC-groups/ATC1.csv", index_col=0)
df_atc2 = pd.read_csv("data/processed/ATC-groups/ATC2.csv", index_col=0)
df_atc3 = pd.read_csv("data/processed/ATC-groups/ATC3.csv", index_col=0)
df_atc4 = pd.read_csv("data/processed/ATC-groups/ATC4.csv", index_col=0)
df_atc5 = pd.read_csv("data/processed/ATC-groups/ATC5.csv", index_col=0)

In [381]:
# an example, other datasets following same pattern
df_atc1.head()

,NAZIV,joint_name
ATC1,,
A,ZDRAVILA ZA BOLEZNI PREBAVIL IN PRESNOVE,A - ZDRAVILA ZA BOLEZNI PREBAVIL IN PRESNOVE
B,ZDRAVILA ZA BOLEZNI KRVI IN KRVOTVORNIH ORGANOV,B - ZDRAVILA ZA BOLEZNI KRVI IN KRVOTVORNIH OR...
C,ZDRAVILA ZA BOLEZNI SRCA IN OŽILJA,C - ZDRAVILA ZA BOLEZNI SRCA IN OŽILJA
D,ZDRAVILA ZA BOLEZNI KOŽE IN PODKOŽNEGA TKIVA,D - ZDRAVILA ZA BOLEZNI KOŽE IN PODKOŽNEGA TKIVA
G,ZDRAVILA ZA BOLEZNI SEČIL IN SPOLOVIL TER SPOL...,G - ZDRAVILA ZA BOLEZNI SEČIL IN SPOLOVIL TER ...


In [304]:
# custom function for assigning proper ATC names
def assign_atc(x, atc_level):
    """
    Returns ATC name depending on the atc abbreviation and
    required atc level
    Args:
        x: str
        atc_level: int
    Returns:
        str 
    """
    if atc_level == 1:
        ref_df = df_atc1
    if atc_level == 2:
        ref_df = df_atc2
    if atc_level == 3:
        ref_df = df_atc3
    if atc_level == 4:
        ref_df = df_atc4
    
    if atc_level == 5:
        ref_df = df_atc5
        
        # if atc 5 won't work we try again with atc4         
        try:
            result = ref_df.loc[x]["joint_name"]
        except KeyError as e:
            ref_df = df_atc4
            try:
                result = ref_df.loc[x]["joint_name"]
            except KeyError as e:
                result = np.nan
                
            return result
            
    
    try:
        result = ref_df.loc[x]["joint_name"]
    except KeyError as e:
        result = np.nan
        
    return result

### Assign ATC names to columns

In [386]:
# assign proper names to atc levels
df_stacked["ATC1"] = df_stacked.AtcOznaka.str[0].apply(assign_atc, atc_level=1)
df_stacked["ATC2"] = df_stacked.AtcOznaka.str[0:3].apply(assign_atc, atc_level=2)
df_stacked["ATC3"] = df_stacked.AtcOznaka.str[0:4].apply(assign_atc, atc_level=3)
df_stacked["ATC4"] = df_stacked.AtcOznaka.str[0:5].apply(assign_atc, atc_level=4)
df_stacked["ATC5"] = df_stacked.AtcOznaka.str[0:7].apply(assign_atc, atc_level=5)

In [387]:
# lets explore if any missing ATCs are present
missing_atc_df = df_stacked.loc[df_stacked[["ATC1", "ATC2", "ATC3", "ATC4", "ATC5"]].isna().any(axis="columns")]

In [388]:
# lets check which medicines are without ATC
missing_atc_df.drop_duplicates(subset=["NacionalnaŠifra", "ImeZdravila", "PoimenovanjeZdravila", "AtcOznaka"])

,NacionalnaŠifra,ImeZdravila,PoimenovanjeZdravila,KratkoImeZdravila,SplosnoImeZdravila,AtcOznaka,KolicinaOEZzaAplikacijo,OznakaOEZ,DDD,DDD_OznakaOEZ,...,Stevilo DDD,Stevilo DID,Vrednost OZZ,Število receptov,Število škatel,ATC1,ATC2,ATC3,ATC4,ATC5
35700,159586,Skyrizi,Skyrizi 75 mg raztopina za injiciranje v napol...,Skyrizi 75 mg razt.za inj. brizga 2x,risankizumab,/,2,brizga,NaN,NaN,...,0.0,0.0,0.00,0,0,NaN,NaN,NaN,NaN,NaN
38680,601013,DEKSPANTENOL RAZT. 100 ML (GORIŠKA LEKARNA NOV...,Dekspantenol raztopina 100 ml (Goriška lekarna...,Dekspantenol raztopina 100 ml (Goriška lekarna...,dekspantenol,NaN,100,ml,NaN,NaN,...,0.0,0.0,0.00,0,0,NaN,NaN,NaN,NaN,NaN
38750,601152,VORIKONAZOL,"Vorikonazol 1% kapljice za oko, raztopina",Vorikonazol 1% kapljice za oko razt. 10 ml,vorikonazol,NaN,10,g,NaN,NaN,...,0.0,0.0,0.00,0,0,NaN,NaN,NaN,NaN,NaN
38960,610046,NaN,Fiziološka raztopina sterilna 250 ml,Fiziološka raztopina sterilna dermal.razt. 250 ml,natrijev klorid,NaN,1,steklenica,NaN,NaN,...,0.0,0.0,1554.78,262,450,NaN,NaN,NaN,NaN,NaN
38970,610054,NaN,Fiziološka raztopina sterilna 500 ml,Fiziološka raztopina sterilna dermal.razt. 500 ml,natrijev klorid,NaN,1,steklenica,NaN,NaN,...,0.0,0.0,5593.69,638,1242,NaN,NaN,NaN,NaN,NaN


In [389]:
# because only small amount or no packages at all were present on the market 
# (except Fiziološka raztopina - not interesting), we will remove these

df_stacked = df_stacked[~df_stacked.NacionalnaŠifra.isin(missing_atc_df.NacionalnaŠifra.unique())]

In [410]:
df_stacked.columns

Index(['NacionalnaŠifra', 'ImeZdravila', 'PoimenovanjeZdravila',
       'KratkoImeZdravila', 'SplosnoImeZdravila', 'AtcOznaka',
       'KolicinaOEZzaAplikacijo', 'OznakaOEZ', 'DDD', 'DDD_OznakaOEZ',
       'Število_DDD_v_pakiranju', 'leto', 'Celotna vrednost receptov',
       'Stevilo DDD', 'Stevilo DID', 'Vrednost OZZ', 'Število receptov',
       'Število škatel', 'ATC1', 'ATC2', 'ATC3', 'ATC4', 'ATC5'],
      dtype='object')

# Add synthetic features

1. Add `percent_insurance_coverage`

Defined as percent of `Vrednost OZZ` from `Celotna vrednost receptov`

In [426]:
df_stacked = df_stacked.assign(
    percent_insurance_coverage= lambda x:  round(x["Vrednost OZZ"] * 100 / x["Celotna vrednost receptov"])
)

2. Add `mean_prescription_value`

Defined as ratio between `Celotna vrednost receptov` and `Število receptov`



In [432]:
df_stacked = df_stacked.assign(
    mean_prescription_value= lambda x:  round(x["Celotna vrednost receptov"] / x["Število receptov"])
)

3. Add `mean_prescription_package_count`

Defined as ratio between `Celotna vrednost receptov` and `Število škatel`


In [446]:
df_stacked = df_stacked.assign(
    mean_prescription_package_count= lambda x:  round(x["Celotna vrednost receptov"] / x["Število škatel"])
)

In [448]:
df_stacked.columns

Index(['NacionalnaŠifra', 'ImeZdravila', 'PoimenovanjeZdravila',
       'KratkoImeZdravila', 'SplosnoImeZdravila', 'AtcOznaka',
       'KolicinaOEZzaAplikacijo', 'OznakaOEZ', 'DDD', 'DDD_OznakaOEZ',
       'Število_DDD_v_pakiranju', 'leto', 'Celotna vrednost receptov',
       'Stevilo DDD', 'Stevilo DID', 'Vrednost OZZ', 'Število receptov',
       'Število škatel', 'ATC1', 'ATC2', 'ATC3', 'ATC4', 'ATC5',
       'percent_insurance_coverage', 'mean_prescription_value',
       'mean_package_count', 'mean_prescription_package_count'],
      dtype='object')

# Short analysis to validate results

In [451]:
df_stacked.groupby(["leto", "ATC1"])["Stevilo DDD", "Stevilo DID"].agg({"sum"}).sort_values(by=["leto", ("Stevilo DDD", "sum")], ascending=False).T

/Users/rok-mi/my_projects/poraba-zdravil/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


leto                                              2019  \
ATC1            C - ZDRAVILA ZA BOLEZNI SRCA IN OŽILJA   
Stevilo DDD sum                           4.203382e+08   
Stevilo DID sum                           5.592800e+02   

leto                                                          \
ATC1            A - ZDRAVILA ZA BOLEZNI PREBAVIL IN PRESNOVE   
Stevilo DDD sum                                 1.871923e+08   
Stevilo DID sum                                 2.490200e+02   

leto                                                   \
ATC1            N - ZDRAVILA Z DELOVANJEM NA ŽIVČEVJE   
Stevilo DDD sum                          1.141916e+08   
Stevilo DID sum                          1.518600e+02   

leto                                                                 \
ATC1            B - ZDRAVILA ZA BOLEZNI KRVI IN KRVOTVORNIH ORGANOV   
Stevilo DDD sum                                        78941714.36    
Stevilo DID sum                                             105.01    

leto                                           \
ATC1            R - ZDRAVILA ZA BOLEZNI DIHAL   
Stevilo DDD sum                   57136344.72   
Stevilo DID sum                         75.97   

leto                                                                          \
ATC1            G - ZDRAVILA ZA BOLEZNI SEČIL IN SPOLOVIL TER SPOLNI HORMONI   
Stevilo DDD sum                                        52614633.30             
Stevilo DID sum                                              69.97             

leto                                                                \
ATC1            M - ZDRAVILA ZA BOLEZNI MIŠIČNO-SKELETNEGA SISTEMA   
Stevilo DDD sum                                         49064020.0   
Stevilo DID sum                                               65.3   

leto                                                                                                   \
ATC1            H - HORMONSKA ZDRAVILA ZA SISTEMSKO ZDRAVLJENJE - RAZEN SPOLNIH HORMONOV IN INSULINOV   
Stevilo DDD sum                                        20400018.12                                      
Stevilo DID sum                                              27.13                                      

leto                                           \
ATC1            S - ZDRAVILA ZA BOLEZNI ČUTIL   
Stevilo DDD sum                   10496137.05   
Stevilo DID sum                         13.95   

leto                                                            ...  \
ATC1            J - ZDRAVILA ZA SISTEMSKO ZDRAVLJENJE INFEKCIJ  ...   
Stevilo DDD sum                                     9398430.01  ...   
Stevilo DID sum                                          12.40  ...   

leto                                                           2010  \
ATC1            B - ZDRAVILA ZA BOLEZNI KRVI IN KRVOTVORNIH ORGANOV   
Stevilo DDD sum                                        58618807.63    
Stevilo DID sum                                              77.94    

leto                                           \
ATC1            R - ZDRAVILA ZA BOLEZNI DIHAL   
Stevilo DDD sum                   46061648.72   
Stevilo DID sum                         61.26   

leto                                                                \
ATC1            M - ZDRAVILA ZA BOLEZNI MIŠIČNO-SKELETNEGA SISTEMA   
Stevilo DDD sum                                        43200420.19   
Stevilo DID sum                                              57.46   

leto                                                                                                   \
ATC1            H - HORMONSKA ZDRAVILA ZA SISTEMSKO ZDRAVLJENJE - RAZEN SPOLNIH HORMONOV IN INSULINOV   
Stevilo DDD sum                                        12945040.80                                      
Stevilo DID sum                                              17.21                                      

leto                                                            \
ATC1            J - ZDRAVILA ZA SISTEMSKO ZD

### The results match the data provided by ZZZS, I let's export the final dataset

In [452]:
df_stacked.to_csv("data/processed/final_dataset_10_19.csv", encoding="utf-8", index=False)

In [456]:
pt = df_stacked.pivot_table(index="leto", values=["Celotna vrednost receptov", "Število škatel"], aggfunc=sum)

In [473]:
pt["Celotna vrednost receptov"].pct_change()*100

leto
2010         NaN
2011    3.691998
2012   -3.407048
2013   -1.127237
2014   -4.693705
2015    3.186251
2016    4.453229
2017    3.761684
2018    6.812142
2019    4.845809
Name: Celotna vrednost receptov, dtype: float64

In [471]:
(pt["Celotna vrednost receptov"].iloc[1] - pt["Celotna vrednost receptov"].iloc[0]) / pt["Celotna vrednost receptov"].iloc[0]

0.036919975124325446

# 